# ADS - Übung 1 - SS2023 [`@Ingrid Scholl - FH Aachen`](https://www.fh-aachen.de/menschen/scholl)
**Thema**: Stacks, Queues und verkettete Listen (VL) 

In [1]:
// includes
#include <iostream>
#include <list>
#include <stdexcept>
#include <stack>
#include <string>
#include <ctype.h>
#include <sstream>

## Aufgabe 1 (Vertauschen von 2 Elementen)
Gegeben seien jeweils Verkettete Listen (VL) mit impliziten head- und tail-Knoten dh. diese sind mit Daten belegt. Für die leere Liste gilt dann head = nullptr und tail = nullptr. Der erste Knoten der VL ist dann ein Knoten, der in diesem Falle gleichzeitig head und taill ist. Folgende Datenstrukturen sind für ein Element der einfachen und doppelten VL definiert:

In [2]:
// Element in einer einfach VL
class NodeE 
{
public:
    int item;
    NodeE *next;
};

// Element in einer doppelt VL
class NodeD 
{
public:
    int item;
    NodeD *next;
    NodeD *prev;
};

Und folgende Datenstrukturen für eine einfache und doppelte VL: 

In [3]:
// Einfach VL
class ListE 
{
public:
    // Konstruktor
    ListE() : m_length(0), m_head(nullptr), m_tail(nullptr) {};
    ListE(std::initializer_list<int> init) : m_length(0), m_head(nullptr), m_tail(nullptr) 
        { for (const int& item : init) this->add(item); }
    // Copy Konstruktor
    ListE(const ListE& other) : m_length(0), m_head(nullptr), m_tail(nullptr) { *this = other; }
    
    // Dekonstruktor
    ~ListE() 
    {
        while (m_head != nullptr)
        {
            NodeE* tmp = m_head->next;
            delete m_head;
            m_head = tmp;
        }
    }
    
    // Add Methode welche am Ende der Liste einen 
    // Knoten mit der Variable item anfuegt
    void add(const int& item)
    {
        m_length++;
        NodeE* new_node = new NodeE();
        new_node->item = item;
        if (m_tail == nullptr) m_head = new_node;
        else m_tail->next = new_node;
        
        m_tail = new_node;
    }
    
    // Search Methode sucht das Element item in der Liste
    bool search(const int& item) 
    {
        NodeE* search_node = this->search_node(item);
        if (search_node == nullptr) {
            std::cout << "Wert " << item << " ist nicht in der Liste vorhanden!" << std::endl;
            return false;
        }
        else {
            std::cout << "Wert " << item << " wurde in der Liste gefunden!" << std::endl;
            return true;
        }
    }
    
    // Print Methode um Liste auszugeben
    void print() const
    {
        NodeE* current = m_head;
        std::cout << "[";
        while (current != nullptr)
        {
            std::cout << current->item;
            if (current->next != nullptr) 
            {
                std::cout << ", ";
                current = current->next;
            }
            else break;
        }
        std::cout << "]" << std::endl;
    }
    
    // Gibt die Laenge der Liste zurueck
    int length() const { return m_length; }
    
    // remove Methode löscht das Element item in der Liste
    bool remove(const int& item) 
    {
        // Suche den Knoten
        NodeE* parent = nullptr;
        NodeE* del_node = m_head;
        while (del_node != nullptr && del_node->item != item) 
        {
            parent = del_node;
            del_node = del_node->next;
        }
        
        if (del_node != nullptr) {
            // Entferne den Knoten
            if (del_node == m_head) m_head = del_node->next;
            if (del_node == m_tail) m_tail = parent;
            
            if (parent != nullptr) parent->next = del_node->next;

            delete del_node;            

            std::cout << "Element " << item << " erfolgreich aus der Liste entfernt!" << std::endl;
            return true;
        }
        else {
            // Knoten nicht gefunden
            std::cout << "Element " << item << " konnte nicht geloescht werden!" << std::endl;
            return false;
        }
    }
    
    // Swap Methode welche den Knoten an der stelle 
    // Index mit dem naechsten tauscht
    bool swap(const int& index);
    
    // Print Methode welche die Knoten rueckwaerts 
    // (in O(n)) ausgibt. Wird bei Aufgabe 4 implementiert.
    void print_reversed() const;
    
    
    // Loescht alle Elemente aus der Liste
    void clear()
    {
        if (m_head == nullptr) return;
        
        // delete all nodes
        while (m_head != nullptr)
        {
            NodeE* tmp = m_head->next;
            delete m_head;
            m_head = tmp;
        }
        
        // clean
        this->m_head = nullptr;
        this->m_tail = nullptr;
        this->m_length = 0;
    }
        
    // Assignment (Copy) Operator
    ListE& operator=(const ListE& other)
    {
        this->clear();
        
        // copy nodes
        NodeE* current = other.m_head;
        while (current != nullptr) 
        {
            this->add(current->item);
            current = current->next;
        }
        return *this;
    }
private:
    // Hilf Methode fuer print reversed (wenn print reversed rekursiv 
    // implementiert wird)
    void print_reversed_helper(NodeE* node) const;
    
    NodeE* search_node(const int& item) 
    {
        NodeE* tmp = m_head;
        while (tmp != nullptr && tmp->item != item) tmp = tmp->next;
        return tmp;
    }
    
    NodeE* m_head;
    NodeE* m_tail;
    int m_length;
};

In [4]:
// Doppelt VL
class ListD
{
public:
    // Konstruktor
    ListD() : m_length(0), m_head(nullptr), m_tail(nullptr) { };
    ListD(std::initializer_list<int> init) : m_length(0), m_head(nullptr), m_tail(nullptr)
        { for (const int& item : init) this->add(item); }
    // Copy Konstruktor
    ListD(const ListD& other) : m_length(0), m_head(nullptr), m_tail(nullptr) { *this = other; }
    
    // Dekonstruktor
    ~ListD() 
    {
        if (m_head == nullptr) return;
        while (m_head->next != nullptr)
        {
            m_head = m_head->next;
            delete m_head->prev;
        }
        delete m_head;
    }
    
    // Add Methode welche am Ende der Liste einen Knoten mit der 
    // Variable item anfuegt
    void add(const int& item)
    {
        m_length++;
        
        NodeD* new_node = new NodeD();
        new_node->item = item;
        new_node->prev = m_tail;
        
        if (m_tail == nullptr) m_head = new_node;
        else m_tail->next = new_node;
        
        m_tail = new_node;
    }
    
    // Search Methode sucht das Element item in der Liste
    bool search(const int& item) const 
    {
        NodeD * search_node = this->search_node(item);
        if (search_node == nullptr) {
            std::cout << "Wert " << item << " ist nicht in der Liste vorhanden!" << std::endl;
            return false;
        }
        else {
            std::cout << "Wert " << item << " wurde in der Liste gefunden!" << std::endl;
            return true;
        }
    }
    
    // Print Methode um Liste auszugeben
    void print() const
    {
        NodeD* current = m_head;
        std::cout << "[";
        while (current != nullptr)
        {
            std::cout << current->item;
            if (current->next != nullptr) 
            {
                std::cout << ", ";
                current = current->next;
            }
            else break;
        }
        std::cout << "]" << std::endl;
    }
    
    // Print Methode um Liste rueckwaerts auszugeben
    void print_reversed() const
    {
        NodeD* current = m_tail;
        std::cout << "[";
        while (current != nullptr)
        {
            std::cout << current->item;
            if (current->prev != nullptr) 
            {
                std::cout << ", ";
                current = current->prev;
            }
            else break;
        }
        std::cout << "]" << std::endl;
    }
    
    // gibt die Laenge der Liste zurueck
    int length() const { return m_length; }
    
    // remove Methode löscht das Element item in der Liste
    bool remove(const int& item) 
    {
        NodeD* del_node = search_node(item);
        if (del_node != nullptr) {
            if (del_node == m_head) m_head = del_node->next;
            if (del_node == m_tail) m_tail = del_node->prev;
            
            if (del_node->prev != nullptr) del_node->prev->next = del_node->next;
            if (del_node->next != nullptr) del_node->next->prev = del_node->prev;

            delete del_node;            

            std::cout << "Element " << item << " erfolgreich aus der Liste entfernt!" << std::endl;
            return true;
        }
        else {
            std::cout << "Element " << item << " konnte nicht geloescht werden!" << std::endl;
            return false;
        }
    }
    
    // Swap Methode welche den Knoten an der stelle 
    // Index mit dem naechsten tauscht
    bool swap(const int& index);
    
    // Loescht alle Elemente aus der Liste
    void clear()
    {
        if (m_head == nullptr) return;
        // delete all nodes
        while (m_head->next != nullptr)
        {
            m_head = m_head->next;
            delete m_head->prev;
        }
        delete m_head;
        
        // cleanup
        this->m_head = nullptr;
        this->m_tail = nullptr;
        this->m_length = 0;
    }
    
    // Assignment (Copy) Operator
    ListD& operator=(const ListD& other)
    {
        this->clear();
        // copy nodes
        NodeD* current = other.m_head;
        while (current != nullptr) 
        {
            this->add(current->item);
            current = current->next;
        }
        return *this;
    }
private:
    NodeD* search_node(const int& item) const
    {
        NodeD* tmp = m_head;
        while (tmp != nullptr && tmp->item != item) tmp = tmp->next;
        return tmp;
    }
    
    NodeD* m_head;
    NodeD* m_tail;
    int m_length;
};

Schreiben Sie jeweils eine Methode swap, die 2 benachbarte Elemente nur durch Umbiegen der Referenzen vertauscht. (**Beachten Sie**: Es ist keine gültige Lösung, wenn Sie nur die Daten in den benach-barten Knoten tauschen. Überlegen Sie, warum im Allgemeinen das Umbiegen der Zeiger auf die Knoten effizienter ist.)
1. Bei einer einfach verketteten Liste:

In [5]:
bool ListE::swap(const int& index)
{
    // Ihre Loesung hier:
    // Korrekten Knoten suchen:
    if (m_head == nullptr)
    {
        std::cout << "Swap: Index is out of bounds!";
        return false;
    }

    NodeE* current = m_head;
    NodeE* prev = nullptr;
    for (int i = 0; i < index; i++)
    {
        if (current->next != nullptr && current->next->next != nullptr)
        {
            prev = current;
            current = current->next;
        }
        else 
        {
            std::cout << "Swap: Index is out of bounds!";
            return false;
        }
    }

    // swap
    if (current->next == m_tail) m_tail = current;    
    if (prev == nullptr) m_head = current->next;
    else prev->next = current->next;
    prev = current->next;
    current->next = prev->next;
    prev->next=current;
    
    return true;
}

In [6]:
// Teste Sie hier Ihre Methode Swap für eine einfach VL:
ListE listE({0,1,2,3,4,5});
std::cout << "Default: ";
listE.print();
for (int i = 0; i < listE.length() - 1; i++)
{
    listE.swap(i);
    std::cout << "After swap " << i << ": ";
    listE.print();
}

Default: [0, 1, 2, 3, 4, 5]
After swap 0: [1, 0, 2, 3, 4, 5]
After swap 1: [1, 2, 0, 3, 4, 5]
After swap 2: [1, 2, 3, 0, 4, 5]
After swap 3: [1, 2, 3, 4, 0, 5]
After swap 4: [1, 2, 3, 4, 5, 0]


2. Bei einer doppelt verketteten Liste:

In [7]:
bool ListD::swap(const int& index)
{
    // Ihre Loesung hier:
    // Korrekten Knoten suchen:
    if (m_head == nullptr)
    {
        std::cout << "Swap: Index is out of bounds!";
        return false;
    }

    NodeD* current = m_head;
    NodeD* prev = nullptr;
    for (int i = 0; i < index; i++)
    {
        if (current->next != nullptr && current->next->next != nullptr)
        {
            prev = current;
            current = current->next;
        }
        else 
        {
            std::cout << "Swap: Index is out of bounds!";
            return false;
        }
    }
    
    NodeD* old1 = prev;
    NodeD* old2 = current;
    
    // swap
    if (current->next == m_tail) m_tail = current;
    if (prev == nullptr) m_head = current->next;
    else
    {
        current->prev = old1->prev;
        prev->prev = current;
        prev->next = old2->next;
        current->next = prev;
    }
    return true;
}

In [8]:
// Teste Sie hier Ihre Methode Swap für eine doppelt VL:
ListE listE({0,1,2,3,4,5});
std::cout << "Default: ";
listE.print();
for (int i = 0; i < listE.length() - 1; i++)
{
    listE.swap(i);
    std::cout << "After swap " << i << ": ";
    listE.print();
}

Default: [0, 1, 2, 3, 4, 5]
After swap 0: [1, 0, 2, 3, 4, 5]
After swap 1: [1, 2, 0, 3, 4, 5]
After swap 2: [1, 2, 3, 0, 4, 5]
After swap 3: [1, 2, 3, 4, 0, 5]
After swap 4: [1, 2, 3, 4, 5, 0]


## Aufgabe 2 (Mischen von 2 Listen)
Gegeben sind zwei sortierte Listen ohne Duplikate L1 und L2:

In [9]:
std::list<int> l1 = {10, 11, 13, 15, 18, 20, 21};
std::list<int> l2 = {4, 8, 10, 12, 14, 15, 18, 23};

1. Schreiben Sie eine Funktion, die nur mit den grundlegenden Listen-Operationen aus der stl::list, s. [hier](https://cplusplus.com/reference/list/list/)  oder [hier](https://cplusplus.com/reference/list/list/front/) die sortierte Vereinigung von beiden Listen erzeugt. (**Ergebnis für das Beispiel**: L1 ∩ L2 = {4,8,10,10,11,12,13,14,15,15,18,18,20,21,23})

In [10]:
std::list<int> union_of_lists(std::list<int>& l1, 
                              std::list<int>& l2)
{
    // Ihre Loesung hier:  
    std::list<int> l3;
    while (l1.empty() != true && l2.empty() != true)
    {
        if ((l1.front() > l2.front()) || (l1.empty() == true && l2.empty() != true))
        {
            l3.push_back(l2.front());
            l2.pop_front();
        }
        else if ((l1.front() <= l2.front()) || (l1.empty() != true && l2.empty() == true))
        {
            l3.push_back(l1.front());
            l1.pop_front();
        }
    }
    return l3;
}

//missing last entry dk why

In [11]:
// Testen Sie hier Ihre Loesung:
std::list<int> l3 = union_of_lists(l1,l2);
std::cout << "Sorted list: ";
while (l3.empty() != true)
{
    std::cout << " " << l3.front();
    l3.pop_front();
}

Sorted list:  4 8 10 10 11 12 13 14 15 15 18 18 20 21

2. Schreiben Sie eine Funktion, die nur mit den grundlegenden Listen-Operationen aus der STL die Schnittmenge von beiden Listen erzeugt.(Ergebnis für das Beispiel: L1 ∪ L2 = {10,15,18})

In [12]:
// Kernel stürzt ab, konnte den Grund nicht finden

/* std::list<int> intersection_of_lists(std::list<int>& l1, 
                                     std::list<int>& l2)
{
    // Ihre Loesung hier:
    std::list<int> l3;
    while ((l1.empty() != true) || (l2.empty() != true))
    {
        if (l1.front() > l2.front())
        {
            l2.pop_front();
        }
        if (l1.front() == l2.front())
        {
            l3.push_back(l2.front());
            l1.pop_front();
            l2.pop_front();
        }  
        if (l1.front() < l2.front()) 
        {
            l1.pop_front();
        }
    }
    return l3;
}
*/

In [13]:
/*
// Teste fuer Ihre Loesung:
std::list<int> l3 = intersection_of_lists(l1,l2);
std::cout << "Sorted list: ";
while (l3.empty() != true)
{
    std::cout << " " << l3.front();
    l3.pop_front();
}
*/

3. Überlegen Sie sich ihre Laufzeit der Methoden in der O-Notation.
O(n) n=sum of length of both lists

## Aufgabe 3 (Deque)
Eine Deque ist eine Datenstruktur, die aus einer Liste von Datenelementen besteht, und folgende Operationen nur zulässt:

1. `push_front(x)`: Einfügen eines neuen Datenelementes x am Anfang der Deque.
2. `pop_front()`:   Entnahme eines Elementes vom Anfang der Deque.
3. `push_back(x)`:  Einfügen eines neuen Datenelementes x am Ende der Deque.
4. `pop_back()`:    Entnahme des letzten Elementes der Deque.
5. `print_all()`:   Ausgabe aller Elemente vom Anfang bis zum Ende der Deque.

Schreiben Sie Methoden, die diese Operationen der Deque in O(1) ermöglicht (`print_all()` in O(n)). Überlegen Sie sich, mit welcher dynamischen Datenstruktur dies erreicht werden kann.

In [14]:
class Deque
{
public:
    // Konstruktor
    Deque() : m_head(nullptr), m_tail(nullptr) {};
    Deque(std::initializer_list<int> init) : m_head(nullptr), m_tail(nullptr)
        { for (const int& item : init) this->push_back(item); }
    // Hier gehoert eigentlich ein Copy Konstruktor hin
    // (der wird aber fuer uebersichtlicheren Code weggelassen)
    
    // Dekonstruktor
    ~Deque()
    {
        if (m_head == nullptr) return;
        while (m_head->next != nullptr)
        {
            m_head = m_head->next;
            delete m_head->prev;
        }
        delete m_head;
    }
    
    // Zu implementierende Methoden
    void push_front(const int& item);
    int pop_front();
    void push_back(const int& item);
    int pop_back();
    void print_all() const;
    
    // Hier gehoert eigentlich ein Assignment (copy) Operator hin
    // (der wird aber fuer uebersichtlicheren Code weggelassen)
private:
    NodeD* m_head;
    NodeD* m_tail;
};

1. `push_front(x)`: Einfügen eines neuen Datenelementes x am Anfang der Deque:

In [15]:
void Deque::push_front(const int& item)
{
    // Ihre Loseung hier:
    if (m_head == nullptr)
    {
        NodeD* new_node = new NodeD();
        new_node-> item = item;
        m_head = new_node;
        m_tail = new_node;
    }
    else if (m_head == m_tail)
    {
        NodeD* new_node = new NodeD();
        new_node-> item = item;
        m_head = new_node;
        m_head->next = m_tail;
        m_tail->prev = m_head;
    }
    else
    {
        NodeD* new_node = new NodeD();
        new_node->item = item;
        m_head->prev = new_node;
        NodeD* old = m_head;
        m_head = new_node;
        m_head->next = old;
    }
}

2. `pop_front()`: Entnahme eines Elementes vom Anfang der Deque:

In [16]:
int Deque::pop_front()
{
    // Ihre Loseung hier:
    if (m_head == nullptr)
    {
        std::cout << "pop_front: Liste ist schon leer!";
        return 0;
    }
    else if (m_head == m_tail)
    {
        NodeD* old = m_head;
        m_head = nullptr;
        m_tail = nullptr;
        return old->item;
    }
    else
    {
        NodeD* old = m_head;
        m_head = m_head->next;
        m_head->prev = nullptr;
        return old->item;
    }
}

3. `push_back(x)`: Einfügen eines neuen Datenelementes x am Ende der Deque:

In [17]:
void Deque::push_back(const int& item)
{
    // Ihre Loseung hier:
     if (m_head == nullptr)
    {
        NodeD* new_node = new NodeD();
        new_node-> item = item;
        m_head = new_node;
        m_tail = new_node;
    }
    else if (m_head == m_tail)
    {
        NodeD* new_node = new NodeD();
        new_node-> item = item;
        m_tail = new_node;
        m_tail->prev = m_head;
        m_head->next = m_tail;
    }
    else
    {
        NodeD* new_node = new NodeD();
        new_node->item = item;
        m_tail->next = new_node;
        NodeD* old = m_tail;
        m_tail = new_node;
        m_tail->prev = old;
    }
}


4. `pop_back()`: Entnahme des letzten Elementes der Deque:

In [18]:
int Deque::pop_back()
{
     // Ihre Loseung hier:
    if (m_head == nullptr)
    {
        std::cout << "pop_front: Liste ist schon leer!";
        return 0;
    }
    else if (m_head == m_tail)
    {
        NodeD* old = m_tail;
        m_head = nullptr;
        m_tail = nullptr;
        return m_tail->item;
    }
    else
    {
        NodeD* old = m_tail;
        m_tail = m_tail->prev;
        m_tail->next = nullptr;
        return m_tail->item;
    }
}

5. `print_all()`: Ausgabe aller Elemente vom Anfang bis zum Ende der Deque:

In [19]:
void Deque::print_all() const
{
    // Ihre Loseung hier:
    std::cout << "print_all: ";
    NodeD* current = m_head;
    std::cout << "[";
    while (current != nullptr)
    {
        std::cout << current->item;
        std::cout << ", ";
        current = current->next;
    }
    std::cout << "]" << std::endl;
}

Testen Sie Ihre Methoden:

In [20]:
// Ihre Loesung hier:
Deque deque1({0,1,2,3,4,5});
std::cout << "Default: ";
deque1.print_all();
deque1.push_front(3);
std::cout << "push_front(3): ";
deque1.print_all();
deque1.push_back(6);
std::cout << "push_back(6): ";
deque1.print_all();
deque1.pop_front();
std::cout << "pop_front(): ";
deque1.print_all();
deque1.pop_back();
std::cout << "pop_back(): ";
deque1.print_all();

Default: print_all: [0, 1, 2, 3, 4, 5, ]
push_front(3): print_all: [3, 0, 1, 2, 3, 4, 5, ]
push_back(6): print_all: [3, 0, 1, 2, 3, 4, 5, 6, ]
pop_front(): print_all: [0, 1, 2, 3, 4, 5, 6, ]
pop_back(): print_all: [0, 1, 2, 3, 4, 5, ]


## Aufgabe 4 (Ausgabe einer 1-fach VL in umgekehrter Reihenfolge)
Schreiben Sie einen Algorithmus, der in umgekehrter Reihenfolge die Elemente einer einfach verketteten Liste ausgibt. Ihr Algorithmus sollte dabei die Laufzeit von O(n) nicht übersteigen. 

Es gibt 2 Lösungen: eine iterative und eine rekursive Lösung. Bei der iterativen Lösung müssen Sie eine weitere Datenstruktur nutzen.

In [21]:
void ListE::print_reversed() const
{
    // Ihre Loesung hier:
}

Wenn der Algorithmus rekursiv implementiert wird, dann kann noch folgende Methode zur Hilfe genommen werden:

In [22]:
void ListE::print_reversed_helper(NodeE* node) const
{
    // Ihre Loesung hier:
}

Testen Sie Ihre Methode(n) hier:

In [23]:
// Ihre Loesung hier:

## Aufgabe 5 (2 Stacks mit einem Array)
Schreiben Sie eine Klasse, die 2 Stacks mit einem Array implementiert. Ihre Stack Methoden sollen keinen Overflow deklarieren obwohl möglicherweise jede Arrayposition schon belegt ist. Überlegen Sie, was Sie tun können.

In [24]:
class DoubleStack
{
public:
    // Ihr(e) Konstruktor(en) hier:  
    
    // Ihr(e) Dekonstruktor(en) hier:
    
    // Fuegt ein Element zu Stack 0 hinzu
    void push0(const int& item);
    // Fuegt ein Element zu Stack 1 hinzu
    void push1(const int& item);
    
    // Entfernt ein Element von Stack 0
    int pop0();
    // Entfernt ein Element von Stack 1
    int pop1();
    
    // Hier gehoert eigentlich ein Assignment (copy) Operator hin
    // (der wird aber fuer uebersichtlicheren Code weggelassen)
private:
    // Ihre Variablen hier:
    int* m_array;
    int m_allocation_size;  // Wie viel Speicher allokiert werden soll, 
                            // bei einer vergroeßerung des Arrays
    int m_size; // Groeße des Arrays
    int m_i_stack0;  // Index von Stack 0
    int m_i_stack1;  // Index von Stack 1
    
    // Interne Resize Methode um bei vollem Array mehr 
    // Speicher zu allokieren:
    void resize();
};

In [25]:
void DoubleStack::push0(const int& item)
{
    // Ihre Loesung hier:
}

In [26]:
void DoubleStack::push1(const int& item)
{
    // Ihre Loesung hier:
}

In [27]:
int DoubleStack::pop0()
{
    // Ihre Loesung hier:
    return 0;
}

In [28]:
int DoubleStack::pop1()
{
    // Ihre Loesung hier:
    return 0;
}

In [29]:
void DoubleStack::resize()
{
    // Ihre Loesung hier:
}

Testen Sie ihre Klasse:

In [30]:
// Ihre Loesung hier:

## Aufgabe 6 (Taschenrechner)
1. Schreiben Sie einen Algorithmus, der einen mathematischen Ausdruck von der Standard-Konsole in Infix-Notation einliest und auf richtige Klammerungen überprüft. Annahme: Zahlen sind nur vom Datentyp int und double. Zur Vereinfachung werden nur runde Klammer auf `(` und zu `)` zugelassen. Operanden sind `+`, `-`, `*`, `/`. *Ist der Ausdruck nicht korrekt, soll eine entsprechende Fehler-Notation ausgegeben werden. Ist der Ausdruck korrekt soll dieser mit der Teilaufgabe 2. weiter verarbeitet werden.*

In [31]:
// Algorithmus zum extrahieren einer Zahl aus einem String
std::string extract_num(const std::string& input, const int& start_index)
{
   std::string number = "";
    bool has_decimal_point = false;
    
    for (int i = start_index; i < input.size(); i++)
    {
        const char& c = input[i];
        if (isdigit(c))number += c;
        else if (c == '.')
        {
            if (has_decimal_point) break;
            else 
            {
                has_decimal_point = true;
                number += c;
            }
        }
        else break;
    }
    return number;
}

In [32]:
// Holen Sie sich hier die eingabe des Nutzers:
std::string input = "(2*3)+(4-1)";

In [33]:
//Algorithmus zum testen einer korrekten Infixnotations Klammerung
bool test_infix_brackets(std::string input)
{
    // Ihre Loesung hier:
   int open_brackets = 0;
    for (char& c : input)
    {
        if (c == '(') open_brackets++;
        else if(c == ')') open_brackets--;
    }
    return open_brackets == 0;
}

In [34]:
// Testen Sie hier Ihren Algorithmus zur kontrolle der Klammerung:
std::cout << "Klammerung der Eingabe ist " 
    << (test_infix_brackets(input) ? "" : "nicht") << "in Ordnung.";

Klammerung der Eingabe ist in Ordnung.

2. Falls der Ausdruck korrekt geklammert ist, soll dieser von der Infix-Notation in die Postfix-Notation umgewandelt werden. 

In [35]:
// Algorithmus zur Umwandlung einer Infix-Notation in eine Postfix-Notation
std::string infix_to_postfix(const std::string& input)
{
   // Ihre Loesung hier:
    std::stack<char> operators;
    std::stringstream term;
    
    // Iteriere durch die Infix-Notation
    for (int i = 0; i < input.size(); i++)
    {
        const char& c = input[i];
        
        // Verarbeiten des chars
        if (c == ' ' or c == '(') continue;
        else if (c == ')')
        {
            // Eine Klammer zu schließt fuegt alle Operatoren an
            while (!operators.empty())
            {
                term << operators.top() << " ";
                operators.pop();
            }
        }
        else if (c == '+' || c == '-' || c == '*' || c == '/') operators.push(c);
        else if (isdigit(c))
        {
            std::string number = extract_num(input, i);
            term << number << " ";  // Zahl direkt ausgeben
            i += number.size() - 1;  // index entsprechend der Groeße von der Zahl verschieben 
        }
        else
        {
            throw std::invalid_argument("Die Infixnotation '" + input + "', hat am Index " 
                                        + std::to_string(i) + " ein ungueltiges Zeichen.");
        }
    }
    
    // restliche Operatoren anfuegen
    while (!operators.empty())
    {
        term << operators.top() << " ";
        operators.pop();
    }
    
    // Rueckgabe der fertigen Postfix Notation als String
    return term.str();
}

In [36]:
// Testen Sie Ihre Funktion:
std::cout << "Infix: " << input << std::endl;
std::cout << "Postfix: " << infix_to_postfix(input) << std::endl;

Infix: (2*3)+(4-1)
Postfix: 2 3 * 4 1 - + 


3. Berechnen Sie den Postfix-Ausdruck mit Hilfe eines Stapels. 
Folgende 2 Regeln sollen angewendet werden: 
    - Push, falls eine Zahl eingelesen wird.
    - Bei Operand: Entnehme 2 Einträge (pop-Operation) vom Stapel, führe die Operation aus. Pushe das Ergebnis wieder in den Stapel.

In [37]:
double calculate_postfix(const std::string& input)
{
    // Ihre Loesung hier
    std::stack<double> numbers;
    
    for (int i = 0; i < input.size(); i++)
    {
        const char& c = input[i];
        
        if (c == ' ') continue;
        else if (isdigit(c))  // 1. Fall
        {
            std::string number = extract_num(input, i);
            numbers.push(std::stod(number));
            i += number.size() - 1;
        }
        else  // 2. Fall
        {
            double num1 = numbers.top();
            numbers.pop();
            double num0 = numbers.top();
            numbers.pop();
            if (c == '+') numbers.push(num0 + num1);
            if (c == '-') numbers.push(num0 - num1);
            if (c == '*') numbers.push(num0 * num1);
            if (c == '/') numbers.push(num0 / num1);
        }
    }
    return numbers.top();
}

Geben Sie den Infix-Ausdruck, den Postfix-Ausdruck und das Endergebnis auf der Konsole aus.

In [38]:
// Testen Sie Ihre Funktion:
std::string input_postfix = infix_to_postfix(input);
double result = calculate_postfix(input_postfix);
std::cout << "Infix: " << input << std::endl;
std::cout << "Postfix: " << input_postfix << std::endl;
std::cout << "Ergebnis: " << result << std::endl;

Infix: (2*3)+(4-1)
Postfix: 2 3 * 4 1 - + 
Ergebnis: 9
